In [1]:
import os
import warnings
import random
from os.path import join
import torch
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
torch.__version__

'1.9.0'

In [3]:
import torchvision

In [4]:
torchvision.__version__

'0.10.0a0'

In [5]:
torch.cuda.is_available()

True

In [6]:
from utils import load_data_sequential
from utils import eval_sequential
import warnings
import random
from os.path import join
import torch
import numpy as np
warnings.filterwarnings('ignore')

# set seed
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

seed = 1
seed_everything(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
top_k = 20

Using backend: pytorch


In [7]:
torch.cuda.is_available()

True

In [8]:
device

device(type='cuda')

In [9]:
import pickle
with open('../output/playlist.data', 'rb') as f:
    playlists = pickle.load(f)

In [10]:
# playlists = playlists[:100000]

In [11]:

num_users = len(playlists)
num_items = 70229

print(f"# of users: {num_users},  # of items: {num_items}")

music_data = dict()
for idx, i in enumerate(playlists):
    music_data[idx] = i
    
    
user_train_music = dict()
user_valid_music = dict()
user_test_music = dict()

for u in music_data:
    user_valid_music[u] = [music_data[u][-2]]
    user_test_music[u] = [music_data[u][-1]]
    user_train_music[u] = music_data[u][:-2]

# of users: 970877,  # of items: 70229


In [12]:
top_k = 500


In [15]:
from models.TransRec_sequential import TransRec_sequential

seed_everything(seed)
TransRec = TransRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, emb_dim=200, maxlen=1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
TransRec.fit()
TransRec_ndcg, TransRec_recall = eval_sequential(TransRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[TransRec]\t Test_Recall@{top_k} = {TransRec_recall:.4f} Test_NDCG@{top_k} = {TransRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in TransRec.parameters() if p.requires_grad))
print("======================================")

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.95it/s]


epoch 1 train_loss = 0.00004675 valid_recall@500 = 0.1430 valid_ndcg@500 = 0.0255


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.17it/s]


epoch 2 train_loss = 0.00000057 valid_recall@500 = 0.1340 valid_ndcg@500 = 0.0210


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.08it/s]


...

epoch 5 train_loss = 0.00000002 valid_recall@500 = 0.1410 valid_ndcg@500 = 0.0216


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 113.21it/s]


Early stop at epoch:6


test: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 970877/970877 [2:36:47<00:00, 103.20it/s]



[TransRec]	 Test_Recall@500 = 0.1348 Test_NDCG@500 = 0.0222
parmas : 208291629


In [16]:
topk = 500

In [17]:
"""
SASRec
"""
from models.SASRec_sequential import SASRec_sequential

seed_everything(seed)
SASRec = SASRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden_dim=50, maxlen=20, num_blocks=1, num_heads=1,
                            num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
SASRec.fit()
SASRec_ndcg, SASRec_recall = eval_sequential(SASRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, 20, mode='test')
print(f"\n[SASRec]\t Test_Recall@{top_k} = {SASRec_recall:.4f} Test_NDCG@{top_k} = {SASRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in SASRec.parameters() if p.requires_grad))

valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 97.61it/s]


epoch 1 train_loss = 0.9672 valid_recall@500 = 0.1460 valid_ndcg@500 = 0.0252


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 97.66it/s]


epoch 2 train_loss = 0.8791 valid_recall@500 = 0.1860 valid_ndcg@500 = 0.0303


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 97.92it/s]


...

epoch 38 train_loss = 0.3424 valid_recall@500 = 0.5770 valid_ndcg@500 = 0.1312


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 97.27it/s]


Early stop at epoch:39


test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 970877/970877 [2:56:06<00:00, 91.88it/s]


[SASRec]	 Test_Recall@500 = 0.1075 Test_NDCG@500 = 0.0447
parmas : 3528100


In [13]:
"""
BERT4Rec
"""
from models.BERTRec_sequential import BERTRec_sequential

seed_everything(seed)
BERTRec = BERTRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden=50*5, maxlen=50, n_layers=2, heads=5, mask_prob=0.1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
BERTRec.fit()
BERTRec_ndcg, BERTRec_recall = eval_sequential(BERTRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[BERTRec]\t Test_Recall@{top_k} = {BERTRec_recall:.4f} Test_NDCG@{top_k} = {BERTRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in BERTRec.parameters() if p.requires_grad))

valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 86.05it/s]


epoch 1 train_loss = 8.1149 valid_recall@500 = 0.3570 valid_ndcg@500 = 0.0658


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.51it/s]


epoch 2 train_loss = 7.9340 valid_recall@500 = 0.4260 valid_ndcg@500 = 0.0839


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.98it/s]


...

epoch 24 train_loss = 7.0668 valid_recall@500 = 0.5420 valid_ndcg@500 = 0.1316


valid: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.73it/s]


Early stop at epoch:25


test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 970877/970877 [3:12:05<00:00, 84.24it/s]


[BERTRec]	 Test_Recall@500 = 0.4834 Test_NDCG@500 = 0.1054
parmas : 36704480


In [ ]:
train_rows = []
for user in user_train_music:
    for song in user_train_music[user]:
        train_rows.append((user,song,1))
train = pd.DataFrame(train_rows, columns = ['user_id', 'item_id', 'rating'])

In [ ]:
valid_rows = []
for user in user_valid_music:
    for song in user_valid_music[user]:
        valid_rows.append((user,song,1))
valid = pd.DataFrame(valid_rows, columns = ['user_id', 'item_id', 'rating'])

In [ ]:
test_rows = []
for user in user_test_music:
    for song in user_test_music[user]:
        test_rows.append((user,song,1))
test = pd.DataFrame(test_rows, columns = ['user_id', 'item_id', 'rating'])

In [ ]:
train = train.to_numpy()
valid = valid.to_numpy()
test = test.to_numpy()

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in train:
    matrix[u, i] = r
train = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in valid:
    matrix[u, i] = r
valid = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in test:
    matrix[u, i] = r
test = sparse.csr_matrix(matrix)

# train = train.toarray()
# valid = valid.toarray()
# test = test.toarray()

In [ ]:
train = train+valid

In [ ]:
top_k = 20

In [ ]:
"""
Embarrassingly shallow autoencoders for sparse data, 
Harald Steck,
Arxiv.
"""
import os
import math
import numpy as np

class EASE_implicit():
    def __init__(self, train, valid, reg_lambda):
        self.train = train.toarray()
        self.valid = valid.toarray()
        self.num_users = train.shape[0]
        self.num_items = train.shape[1]
        self.reg_lambda = reg_lambda

    def fit(self):   
        train_matrix = self.train
        G = train_matrix.T @ train_matrix
        diag = np.diag_indices(G.shape[0]) 
        G[diag] += self.reg_lambda 
        P = np.linalg.inv(G)
        

        self.enc_w = P / (-np.diag(P)) 
        self.enc_w[diag] = 0


        self.reconstructed = self.train @ self.enc_w

    def predict(self, user_id, item_ids):
        return self.reconstructed[user_id, item_ids]

In [ ]:
"""
EASE
"""
# from models.EASE_implicit import EASE_implicit
from utils import eval_implicit
seed_everything(seed)
ease = EASE_implicit(train=train, valid=valid, reg_lambda=100)
ease.fit()

In [ ]:
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train.toarray(), test.toarray(), 50)
print("EASE Prec@50: %f, Recall@50: %f, NDCG@50: %f"%(ease_prec, ease_recall, ease_ndcg))

In [ ]:
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train.toarray(), test.toarray(), 500)
print("EASE Prec@100: %f, Recall@100: %f, NDCG@100: %f"%(ease_prec, ease_recall, ease_ndcg))